# Model D Validation Notebook

This notebook validates the Model D pipeline end-to-end:
- Trend stability from `fundamentals_history` or `features_fundamental_trends`.
- Beta distribution from price returns.
- Factor correlation heatmap for derived trend features.

It is safe to run even if tables are missing; the notebook will print guidance.


In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text

warnings.filterwarnings("ignore")

DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    raise SystemExit("DATABASE_URL not set")

engine = create_engine(DATABASE_URL)

def table_exists(table_name: str) -> bool:
    with engine.connect() as conn:
        result = conn.execute(text("select to_regclass(:t)"), {"t": table_name})
        return result.scalar() is not None

print("Connected to database.")


In [ ]:
# Load tables if available
fundamentals_history = None
features_trends = None
prices = None

if table_exists("fundamentals_history"):
    fundamentals_history = pd.read_sql("select ticker, as_of, metric, value from fundamentals_history", engine)
    print(f"fundamentals_history rows: {len(fundamentals_history):,}")
else:
    print("fundamentals_history not found.")

if table_exists("features_fundamental_trends"):
    features_trends = pd.read_sql(
        "select ticker, metric, window, mean_value, pct_change, slope, volatility, as_of "
        "from features_fundamental_trends",
        engine,
    )
    print(f"features_fundamental_trends rows: {len(features_trends):,}")
else:
    print("features_fundamental_trends not found.")

if table_exists("prices"):
    prices = pd.read_sql(
        "select dt, symbol, close from prices order by symbol, dt",
        engine,
    )
    print(f"prices rows: {len(prices):,}")
else:
    print("prices not found.")


## 1) Trend stability (slope and percent change)

If `features_fundamental_trends` exists, use it directly. Otherwise, derive simple slopes from
`fundamentals_history` by ticker/metric. This validates whether factor trends are stable and
directional across the universe.

In [ ]:
trend_df = None
if features_trends is not None and not features_trends.empty:
    trend_df = features_trends.copy()
elif fundamentals_history is not None and not fundamentals_history.empty:
    # Simple derivation for validation only
    trend_rows = []
    for (ticker, metric), grp in fundamentals_history.groupby(["ticker", "metric"]):
        grp = grp.sort_values("as_of").tail(8)
        if len(grp) < 3:
            continue
        x = np.arange(len(grp))
        y = grp["value"].astype(float).values
        slope = np.polyfit(x, y, 1)[0]
        pct_change = (y[-1] / y[0]) - 1 if y[0] not in (0, None) else np.nan
        trend_rows.append({
            "ticker": ticker,
            "metric": metric,
            "window": len(grp),
            "mean_value": np.mean(y),
            "pct_change": pct_change,
            "slope": slope,
            "volatility": np.std(np.diff(y)) if len(y) > 1 else np.nan,
            "as_of": grp["as_of"].max(),
        })
    trend_df = pd.DataFrame(trend_rows)
+
if trend_df is None or trend_df.empty:
    print("No trend data available. Populate fundamentals_history or features_fundamental_trends.")
else:
    display(trend_df.head(5))
+
    plt.figure(figsize=(10, 4))
    sns.histplot(trend_df["slope"].dropna(), bins=50, kde=True)
    plt.title("Distribution of Fundamental Trend Slopes")
    plt.xlabel("Slope")
    plt.ylabel("Count")
    plt.show()
+
    plt.figure(figsize=(10, 4))
    sns.histplot(trend_df["pct_change"].dropna(), bins=50, kde=True)
    plt.title("Distribution of Fundamental Percent Change")
    plt.xlabel("Percent Change")
    plt.ylabel("Count")
    plt.show()


## 2) Beta distribution (market proxy)

Use equal-weight market return as a proxy and compute per-ticker beta across the last 252 days.


In [ ]:
if prices is None or prices.empty:
    print("No prices data available for beta calculation.")
else:
    px = prices.copy()
    px["dt"] = pd.to_datetime(px["dt"])
    px = px.sort_values(["symbol", "dt"])
    px["ret1"] = px.groupby("symbol")["close"].pct_change()

    recent = px.groupby("symbol").tail(252)
    pivot = recent.pivot(index="dt", columns="symbol", values="ret1").dropna(how="all")

    market_ret = pivot.mean(axis=1)
    betas = {}
    for symbol in pivot.columns:
        series = pivot[symbol].dropna()
        aligned = series.align(market_ret, join="inner")[0]
        if len(aligned) < 60:
            continue
        mkt = market_ret.loc[aligned.index]
        cov = np.cov(aligned, mkt)[0, 1]
        var = np.var(mkt)
        if var == 0:
            continue
        betas[symbol] = cov / var

    beta_series = pd.Series(betas).dropna()
    print(f"Beta sample size: {len(beta_series)}")

    if not beta_series.empty:
        plt.figure(figsize=(10, 4))
        sns.histplot(beta_series, bins=50, kde=True)
        plt.title("Beta Distribution (252d, equal-weight proxy)")
        plt.xlabel("Beta")
        plt.ylabel("Count")
        plt.show()


## 3) Factor correlation heatmap

Compute cross-metric correlations for the latest trend snapshot.


In [ ]:
if trend_df is None or trend_df.empty:
    print("No trend data available for correlation analysis.")
else:
    latest_as_of = trend_df["as_of"].max()
    latest = trend_df[trend_df["as_of"] == latest_as_of]

    wide = latest.pivot_table(index="ticker", columns="metric", values="pct_change")
    if wide.shape[1] < 2:
        print("Not enough metrics to compute correlations.")
    else:
        corr = wide.corr()
        plt.figure(figsize=(10, 6))
        sns.heatmap(corr, cmap="coolwarm", center=0, square=True)
        plt.title("Factor Correlation (pct_change, latest snapshot)")
        plt.show()


## 4) Summary checks

Use this section to validate coverage and track data completeness.


In [ ]:
summary = {
    "fundamentals_history_rows": int(len(fundamentals_history)) if fundamentals_history is not None else 0,
    "features_trends_rows": int(len(features_trends)) if features_trends is not None else 0,
    "prices_rows": int(len(prices)) if prices is not None else 0,
}
summary
